In [1]:
import pydeck 
import pandas as pd 
import matplotlib.pyplot as plt
import statistics 
from pathlib import Path 
import requests 
import pandas as pd
import time
from datetime import datetime

MAPBOX_API_KEY = "pk.eyJ1Ijoiam5haXIiLCJhIjoiY20zYmNiNzkwMDRoNTJqczc5dHo4YzZnNCJ9.Pb96SMF0Hw5WQoQJumo1hw" 

#### 1. Most frequent travelled routes | Origin-Destination pairs 

Visualization:
- Layer: ArcLayer for pydeck 
- Map  : MapBox  

In [2]:
trip_data_path = Path("./FLEXI_trip_data.xls")
stop_data_path = Path("./FLEXI_bus_stops.xls")

In [3]:
#NOTE: Use relative paths within the folder to avoid path-based conflicts 

%store -r origin_destination_pair 
df_origin_dest = origin_destination_pair

In [5]:
df_origin_dest

,pickup_index,pickup_name,pickup_district,pickup_latitude,pickup_longitude,index_dropoff,name_dropoff,district_dropoff,latitude_dropoff,longitude_dropoff,Frequency
12,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,19,"Beilngries, Ringstrasse",Beilngries,49.033832,11.471982,100
187,19,"Beilngries, Ringstrasse",Beilngries,49.033832,11.471982,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,97
19,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,28,"Beilngries, Neumarkter Stra√üe",Beilngries,49.036912,11.471075,86
275,28,"Beilngries, Neumarkter Stra√üe",Beilngries,49.036912,11.471075,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,85
18,0,"Kinding, Bahnhof",Kinding,48.992168,11.377365,27,"Beilngries, Hafen",Beilngries,49.041112,11.471240,54
...,...,...,...,...,...,...,...,...,...,...,...
154,12,"Badanhausen, Fischerstra√üe",Badanhausen,49.017245,11.444688,23,"Beilngries, An der Leitn",Beilngries,49.030037,11.484332,1
534,64,"Plankstetten, Biberbacher Stra√üe",Biberbach,49.068532,11.455142,11,"Kratzmuehle, Seerestaurant",Pfraundorf,49.004607,11.447705,1
533,64,"Plankstetten, Biberbacher Stra√üe",Biberbach,49.068532,11.455142,1,"Enkering, Maibaum",Enkering,48.992498,11.363953,1
520,59,"Kevenhuell, Kirche",Leising,49.058008,11.516585,31,"Beilngries, Eichstaetter Strasse / Tankstelle",Beilngries,49.035227,11.467885,1


2nd try: Arc's are not working! 

Yay -> ArcLayer works, fine. Need's fine tune on display! 

In [24]:
import streamlit as st 

In [25]:
## BACK TO TUTOTIAL MODE 
# TRIPSLAYER FOR TIME BASED / DEMAND BASED ANIMATION 
# "mapbox://styles/uberdata/cjoqbbf6l9k302sl96tyvka09"
# mapbox://styles/mapbox/light-v9"
# COLLECT THE MINIMUM DATA FOR TESTING 
df_test = df_origin_dest #.nlargest(10, 'Frequency') 
max_frequency = df_test["Frequency"].max()
min_frequency = df_test["Frequency"].min() 
df_test["height"] = df_test["Frequency"] / (max_frequency) * 2
# FIRST CREATE A MAP 

st.title("Trip Vislualization with Frequency threshold")
frequency_threshold = st.slider("Frequency", min_frequency, max_frequency, min_frequency)

filtered_df = df_test[df_test["Frequency"] >= frequency_threshold]

# CREATE A MINIMAL VIEW STATE: 
INITIAL_VIEW_STATE = pydeck.ViewState(
    latitude=df_test["pickup_latitude"].mean(), 
    longitude=df_test["pickup_longitude"].mean(),
    zoom=11, 
    pitch=50 
)

# CREATE A MINIMAL ARCLAYER 
arc_layer = pydeck.Layer(
    "ArcLayer", 
    data=df_test, 
    get_source_position=["pickup_longitude", "pickup_latitude"], 
    get_target_position=["longitude_dropoff", "latitude_dropoff"],
    get_height="height", 
    get_width=20,
    get_tilt=35, 
    get_source_color=[255,0,0, 140], 
    get_target_color=[0,0,255,140],
    pickable=True, 
    auto_highlight=True,
) 

deck = pydeck.Deck(
    layers=[arc_layer],
    initial_view_state=INITIAL_VIEW_STATE,
    map_style="dark"
) 

st.pydeck_chart(deck)


# deck = pydeck.Deck(
#         layers=[trips_layer, buildings_layer],
#         initial_view_state=INITIAL_VIEW_STATE,
#         map_style='dark'
#     )

2024-11-11 00:41:36.908 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 00:41:36.974 
  command:

    streamlit run /home/jnairx/miniconda3/envs/hack/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-11-11 00:41:36.974 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 00:41:36.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 00:41:36.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 00:41:36.976 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 00:41:36.976 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 00:41:36.977 Thr

DeltaGenerator()

In [23]:
deck.show()